In [1]:
import threading
import time
import os

In [3]:
def encodelzw(startindex, endindex, rank):
    global DICTIONARY_SIZE
    string = ''
    compressed_data = []
    data = DATA[startindex:endindex+1]
    print('|' * 10, f'HILO {rank} -> CODIFICANDO {data}')
    for symbol in data:
        string_plus_symbol = string + symbol
        if string_plus_symbol in DICTIONARY:
            string = string_plus_symbol
        else:
            compressed_data.append(DICTIONARY[string])
            if len(DICTIONARY) <= MAXIMUM_SIZE:
                DICTIONARY[string_plus_symbol] = DICTIONARY_SIZE
                DICTIONARY_SIZE += 1
            string = symbol
    if string in DICTIONARY:
        compressed_data.append(DICTIONARY[string])
    print('|' * 10, f'RESULTADO DE HILO {rank} -> {compressed_data}')
    RESULT[rank] = compressed_data

In [4]:
def format_compressed_data(compressed_data):
    output_string = ''
    for i in range(len(compressed_data)):
        rank = compressed_data[i]
        for j in range(len(compressed_data[i])):
            code = rank[j]
            if j < len(compressed_data[i]) - 1:
                output_string += f'{code},'
            else:
                output_string += str(code)
        if i < len(compressed_data) - 1:
            output_string += '\n'
    return output_string

In [6]:
# GLOBAL
start_time = time.time()
BITS = 9

# LEER ARCHIVO
file = open('data.txt', 'r')
DATA = file.read()
# DICCIONARIO SE INICIALIZA CON 256 CARACTERES ASCII
DICTIONARY_SIZE = pow(2,int(BITS-1))
print('~' * 10, f'INICIALIZANDO DICCIONARIO CON {DICTIONARY_SIZE} CARACTERES ASCII')
#DICTIONARY_SIZE = 65536
DICTIONARY = {chr(i): i for i in range(DICTIONARY_SIZE)}  
# CON 9 BITS SE PUEDE TENER UN DICCIONARIO DE HASTA 512 ESPACIOS

MAXIMUM_SIZE = pow(2,int(BITS))

# PARALELIZACION
P = 4  # NUMERO DE HILOS UTILIZADOS
offset = len(DATA) // P  # CARACTER EXTRA
RESULT = [[] for p in range(P)]  # LISTA DE RESULTADOS DE CADA HILO (BIDIMENSIONAL)

print('~' * 10, 'CODIFICANDO TROZOS DEL STRING EN CADA HILO')
for rank in range(P):
    # CALCULO DE LOS RANGOS DEL STRING PARA CADA HILO
    startindex = rank * offset
    if rank == P - 1:
        endindex = len(DATA) - 1
    else:
        endindex = startindex + offset - 1
        
    # EJECUCION PARALELA DE LA FUNCION DE CODIFICACION
    # CON CADA TROZO DEL STRING POR HILO
    args = (startindex, endindex, rank)
    t = threading.Thread(target=encodelzw, args=args)
    t.start()
    t.join()

print('=' * 10, f'RESULTADO DE LA CODIFICACION: {RESULT}')
print('~' * 10, "TIEMPO DE EJECUCION:  %s " % (round((time.time() - start_time),2)*1000), " Mili-segundos")

    
print('~' * 10, 'GUARDANDO STRING CODIFICADO')
# MOSTRAR Y GUARDAR RESULTADO
output = open('encoded_data.txt', 'w+')
output_string = format_compressed_data(RESULT)
output.write(output_string)
output.close()


encodedFile = os.path.getsize('encoded_data.txt')
print('~' * 10, 'TAMAÑO DEL ARCHIVO CODIFICADO: ', encodedFile, "Bytes")

#GUARDAR TIEMPO TXT
size = os.path.getsize('data.txt')
output = open('Tiempos/tiempo_pe-%s.txt' % (size), 'w+')
output_string = str(round((time.time() - start_time),2)*1000)
output.write(output_string)
output.close()

~~~~~~~~~~ INICIALIZANDO DICCIONARIO CON 256 CARACTERES ASCII
~~~~~~~~~~ CODIFICANDO TROZOS DEL STRING EN CADA HILO
|||||||||| HILO 0 -> CODIFICANDO ththega
|||||||||| RESULTADO DE HILO 0 -> [116, 104, 256, 101, 103, 97]
|||||||||| HILO 1 -> CODIFICANDO meinthe
|||||||||| RESULTADO DE HILO 1 -> [109, 101, 105, 110, 258]
|||||||||| HILO 2 -> CODIFICANDO middleo
|||||||||| RESULTADO DE HILO 2 -> [109, 105, 100, 100, 108, 101, 111]
|||||||||| HILO 3 -> CODIFICANDO fthenight
|||||||||| RESULTADO DE HILO 3 -> [102, 258, 110, 105, 103, 257]
========== RESULTADO DE LA CODIFICACION: [[116, 104, 256, 101, 103, 97], [109, 101, 105, 110, 258], [109, 105, 100, 100, 108, 101, 111], [102, 258, 110, 105, 103, 257]]
~~~~~~~~~~ TIEMPO DE EJECUCION:  36120.0   Mili-segundos
~~~~~~~~~~ GUARDANDO STRING CODIFICADO
~~~~~~~~~~ TAMAÑO DEL ARCHIVO CODIFICADO:  94 Bytes
